In [ ]:
#Convolutional Neural Networks

In [ ]:
#Using Pytorch

In [ ]:
import numpy as np
import torch
import os
import pickle
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torch.autograd import Variable

In [ ]:
#Loading the dataset generated using Word2Vec

In [ ]:
split = 0.9

with open('final_data_google.p', 'rb') as f:
	data_ = pickle.load(f)
	idx = np.arange(data_[0].shape[0])
	np.random.shuffle(idx)

	train_data = data_[0][idx[:int(idx.shape[0]*split)]]
	train_truth = data_[1][idx[:int(idx.shape[0]*split)]]
	
	test_data = data_[0][idx[int(idx.shape[0]*split):]]
	test_truth = data_[1][idx[int(idx.shape[0]*split):]]

In [ ]:
#Train dataloader Class

In [ ]:
class train_DataSet(data.Dataset):

	def __init__(self, train_data, train_truth):

		self.train_data = train_data
		self.truth = train_truth

		return

	def __getitem__(self,index):

		return (self.train_data[index][None,:,:], np.argmax(self.truth[index]))

	def __len__(self):

		return len(self.train_data)

In [ ]:
#Test dataloader Class

In [ ]:
class test_DataSet(data.Dataset):

	def __init__(self, test_data, test_truth):

		self.train_data = test_data
		self.truth = test_truth

	def __getitem__(self,index):

		return (self.train_data[index][None,:,:], np.argmax(self.truth[index]))

	def __len__(self):

		return len(self.train_data)

In [ ]:
#Network architecture class

In [ ]:
class Net(nn.Module):
    def __init__(self, first, second):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(300, first, kernel_size=(3,1), padding=(1, 0))
        self.conv2 = nn.Conv2d(first, second, kernel_size=(3,1), padding=(1, 0))
        self.conv3 = nn.Conv2d(second, 444, kernel_size=(1,1), padding=(0, 0))
        self.final = nn.AdaptiveAvgPool2d((1,1))

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.conv3(x)
        x = self.final(x).squeeze().unsqueeze(0)

        return x

In [ ]:
#Initialising To save variable

In [ ]:
all_max_accuracy = np.zeros([len(range(20, 100, 20)), len(range(50, 150, 20))])

In [1]:
#Model architecture varied on grid
#Training and Testing and reporting the results
#Batch size = 10
#Learning Rate fixed to 0.001 (Adaptive Learning rate of Adam Optimizer assures convergence)

In [ ]:
for ____i, first in enumerate(range(20, 100, 20)):
	for ____j, second in enumerate(range(50, 150, 20)):
        net = Net(first , second).cuda()
		trainloader = train_DataSet(train_data, train_truth)
		testloader = test_DataSet(test_data, test_truth)
		print(net)
        batch_size = 10
		criterion = nn.CrossEntropyLoss()
		optimizer = optim.Adam(net.parameters(), lr=0.01/batch_size)
		optimizer.zero_grad()
		steps = 0
		for epoch in range(30):

			running_loss = 0.0
			correct = 0

			net.train()
			
			for i, data in enumerate(trainloader):
				# get the inputs
				inputs, labels = data
				inputs = inputs[None, :, :, :].transpose(0, 3, 2, 1)
				labels = np.array([labels])

				inputs = Variable(torch.from_numpy(inputs).cuda())
				labels = Variable(torch.from_numpy(labels).cuda())

				outputs = net(inputs)
				v, index = torch.max(outputs,1)
				index = index[0].data.cpu().numpy()

				if index == labels.data.cpu().numpy()[0]:
					correct += 1

				loss = criterion(outputs, labels)
				loss.backward()

				if i % batch_size == 0 and i!=0:
					optimizer.step()
					optimizer.zero_grad()
					steps += 1

				running_loss += loss.item()

			print('\n\n--------------------------------------')

			print('Epoch: ', epoch + 1)
			print('Steps: ', steps)
			print('Train accuracy: ', correct/len(trainloader))
			print('Train loss: ', running_loss / len(trainloader))
			
			running_loss = 0.0
			correct = 0

			net.eval()

			for i, data in enumerate(testloader):

				inputs, labels = data
				inputs = inputs[None, :, :, :].transpose(0, 3, 2, 1)
				labels = np.array([labels])

				inputs = Variable(torch.from_numpy(inputs).cuda())
				labels = Variable(torch.from_numpy(labels).cuda())

				outputs = net(inputs)
				v, index = torch.max(outputs,1)
				index = index[0].data.cpu().numpy()

				if index == labels.data.cpu().numpy()[0]:
					correct += 1

				loss = criterion(outputs, labels)

				if i % batch_size == 0 and i!=0:
					optimizer.step()
					optimizer.zero_grad()
					steps += 1
					
				running_loss += loss.item()

			print('Test accuracy = ', correct/len(testloader))
			print('Test loss = ', running_loss/len(testloader))

			if correct/len(testloader) > all_max_accuracy[____i, ____j]:
				all_max_accuracy[____i, ____j] = correct/len(testloader)

print(all_max_accuracy)
np.save('Grid Run results', all_max_accuracy)